# Reclamation Assessment using Z-Score Transformed NDVI (Enhanced)

## Overview
This enhanced notebook analyzes crop growth within lease boundaries compared to the background field using z-score transformation of NDVI rasters. You can now choose between:
- **Robust Z-Score**: Uses median and MAD (resistant to outliers)
- **Normal Z-Score**: Uses mean and standard deviation (better for uniform data)

## Key Features
1. **Automatic Distribution Analysis**: Tests data normality and provides recommendations
2. **Flexible Transformation**: Choose the method that best fits your data
3. **Comparison Preview**: See both methods side-by-side before choosing
4. **Consistent Processing**: Apply the same method across all years for fair comparison

## Workflow
1. Upload NDVI rasters and boundary polygons
2. Analyze data distribution and review recommendations
3. Choose z-score method (or accept recommendation)
4. Process all rasters with selected method
5. Download transformed rasters for GIS analysis

## Formula Reference
- **Robust Z-Score**: `Z = (NDVI - Median) / (1.4826 × MAD)`
- **Normal Z-Score**: `Z = (NDVI - Mean) / Standard_Deviation`

## 1. Setup and Imports

In [ ]:
# Install required packages
%pip install -q geopandas rasterio fiona shapely numpy pandas matplotlib ipywidgets

# Import libraries
import os
import warnings
import zipfile
from datetime import datetime
from typing import List, Tuple, Optional, Dict, Any

import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.io import MemoryFile
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from shapely.geometry import shape, mapping
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Google Colab specific imports
from google.colab import files

warnings.filterwarnings('ignore')

print("✅ Setup complete. All libraries imported successfully.")
print("📍 Running in Google Colab environment")

## 2. File Upload Interface

In [ ]:
# Initialize file storage if not exists
if 'all_uploaded_files' not in locals():
    all_uploaded_files = {}
if 'ndvi_files' not in locals():
    ndvi_files = []
if 'polygon_files' not in locals():
    polygon_files = []

print("📂 Upload your files (you can run this cell multiple times to add more):")
print("="*50)
print("Required files:")
print("1️⃣ NDVI Rasters (.tif or .tiff files)")
print("2️⃣ Field Boundary polygon (.kml, .geojson, or .shp)")
print("3️⃣ Lease Boundary polygon (.kml, .geojson, or .shp)")
print("="*50)

# Upload files
uploaded = files.upload()

# Add to master collection and save to disk
for filename, content in uploaded.items():
    all_uploaded_files[filename] = content
    with open(filename, 'wb') as f:
        f.write(content)
    print(f"✅ Added: {filename} ({len(content)/1024:.1f} KB)")

# Re-categorize all files
ndvi_files = []
polygon_files = []

for filename in all_uploaded_files.keys():
    if filename.lower().endswith(('.tif', '.tiff')):
        ndvi_files.append(filename)
    elif filename.lower().endswith(('.kml', '.geojson', '.shp', '.json')):
        polygon_files.append(filename)

# Sort files for consistent processing
ndvi_files.sort()
polygon_files.sort()

print(f"\n📊 Total Files Summary:")
print(f"   - NDVI rasters: {len(ndvi_files)} files")
print(f"   - Polygon files: {len(polygon_files)} files")

if len(ndvi_files) > 0 and len(polygon_files) >= 2:
    print("\n✅ All required file types present!")
else:
    if len(ndvi_files) == 0:
        print("\n⚠️ Warning: No NDVI rasters uploaded yet")
    if len(polygon_files) < 2:
        print(f"\n⚠️ Warning: Need 2 polygon files (have {len(polygon_files)})")

## 3. Identify Field and Lease Boundaries

In [ ]:
# Helper function to load polygon
def load_polygon(filename: str) -> gpd.GeoDataFrame:
    """Load polygon from various formats"""
    try:
        gdf = gpd.read_file(filename)
        return gdf
    except Exception as e:
        print(f"Error loading {filename}: {e}")
        return None

# Load and identify polygons
if len(polygon_files) >= 2:
    print("🔍 Identifying field and lease boundaries...\n")
    
    # Try to auto-identify based on filename
    field_boundary_file = None
    lease_boundary_file = None
    
    for filename in polygon_files:
        fname_lower = filename.lower()
        if 'field' in fname_lower and not field_boundary_file:
            field_boundary_file = filename
        elif 'lease' in fname_lower and not lease_boundary_file:
            lease_boundary_file = filename
    
    # If not automatically identified, use manual selection
    if not field_boundary_file or not lease_boundary_file:
        field_boundary_file = polygon_files[0]
        lease_boundary_file = polygon_files[1]
    
    print(f"📍 Field Boundary: {field_boundary_file}")
    print(f"📍 Lease Boundary: {lease_boundary_file}")
    
    # Load the polygons
    field_gdf = load_polygon(field_boundary_file)
    lease_gdf = load_polygon(lease_boundary_file)
    
    if field_gdf is not None and lease_gdf is not None:
        print("\n✅ Both boundaries loaded successfully")
else:
    print("❌ Need at least 2 polygon files to proceed")
    field_gdf = None
    lease_gdf = None

## 4. Analyze Data Distribution & Select Z-Score Method

In [ ]:
def calculate_all_stats(data: np.ndarray) -> Dict[str, float]:
    """Calculate both robust and normal statistics"""
    # Remove NaN and NoData values
    valid_data = data[~np.isnan(data)]
    
    # Additional filtering for common NoData values
    nodata_values = [-9999, -10000, -3.4028235e+38, 3.4028235e+38]
    for ndv in nodata_values:
        valid_data = valid_data[np.abs(valid_data - ndv) > 1e-6]
    
    if len(valid_data) == 0:
        return {
            'median': np.nan, 
            'mad': np.nan, 
            'robust_std': np.nan,
            'mean': np.nan,
            'std': np.nan,
            'n_valid': 0
        }
    
    # Calculate robust statistics
    median = np.median(valid_data)
    mad = np.median(np.abs(valid_data - median))
    robust_std = 1.4826 * mad  # Scale factor for consistency with standard deviation
    
    # Calculate normal statistics
    mean = np.mean(valid_data)
    std = np.std(valid_data)
    
    return {
        'median': median,
        'mad': mad,
        'robust_std': robust_std,
        'mean': mean,
        'std': std,
        'n_valid': len(valid_data)
    }

def test_distribution_normality(stats: Dict[str, float]) -> Dict[str, Any]:
    """Test if distribution is suitable for normal z-scores (rule of thumb)"""
    
    if stats['n_valid'] == 0 or np.isnan(stats['mean']):
        return {
            'recommendation': 'robust',
            'reason': 'No valid data',
            'details': {}
        }
    
    # Test 1: Check if mean and median are close (|μ − m| ≤ 0.2σ)
    mean_median_diff = abs(stats['mean'] - stats['median'])
    threshold = 0.2 * stats['std'] if stats['std'] > 0 else 0
    condition1 = mean_median_diff <= threshold
    
    # Test 2: Check if MAD and std are proportional (0.8 ≤ MADs/σ ≤ 1.2)
    if stats['std'] > 0:
        ratio = stats['robust_std'] / stats['std']
        condition2 = 0.8 <= ratio <= 1.2
    else:
        ratio = np.inf
        condition2 = False
    
    # Test 3: Check for extreme uniformity (very small MAD)
    if stats['mad'] < 0.005:  # Extremely uniform data
        return {
            'recommendation': 'normal',
            'reason': '⚠️ HIGH UNIFORMITY DETECTED',
            'warning': 'Data is extremely uniform - normal z-scores recommended',
            'details': {
                'mean_median_diff': mean_median_diff,
                'threshold': threshold,
                'mad_std_ratio': ratio,
                'mad': stats['mad']
            }
        }
    
    # Determine recommendation
    if condition1 and condition2:
        return {
            'recommendation': 'normal',
            'reason': 'Distribution is approximately normal',
            'details': {
                'mean_median_diff': mean_median_diff,
                'threshold': threshold,
                'mad_std_ratio': ratio
            }
        }
    else:
        reasons = []
        if not condition1:
            reasons.append(f'Mean-median difference too large')
        if not condition2:
            reasons.append(f'MAD/Std ratio outside normal range')
        
        return {
            'recommendation': 'robust',
            'reason': 'Distribution appears skewed or contains outliers',
            'details': {
                'mean_median_diff': mean_median_diff,
                'threshold': threshold,
                'mad_std_ratio': ratio,
                'issues': reasons
            }
        }

print("✅ Statistical analysis functions loaded")

In [ ]:
# Analyze each NDVI raster's distribution
distribution_analysis = []

if ndvi_files and field_gdf is not None and lease_gdf is not None:
    print("🔍 Analyzing data distributions...")
    print("="*60)
    
    field_geom = field_gdf.geometry[0]
    lease_geom = lease_gdf.geometry[0]
    
    for i, raster_file in enumerate(ndvi_files):
        print(f"\n[{i+1}/{len(ndvi_files)}] {raster_file}")
        print("-" * 40)
        
        try:
            with rasterio.open(raster_file) as src:
                # Reproject polygons if needed
                raster_crs = src.crs
                if field_gdf.crs != raster_crs:
                    field_geom_proj = field_gdf.to_crs(raster_crs).geometry[0]
                    lease_geom_proj = lease_gdf.to_crs(raster_crs).geometry[0]
                else:
                    field_geom_proj = field_geom
                    lease_geom_proj = lease_geom
                
                # Get field data
                field_data, out_transform = mask(src, [field_geom_proj], crop=True, nodata=np.nan, filled=True)
                field_data = field_data[0].astype(np.float32)
                
                # Create masks
                from rasterio.features import geometry_mask
                field_mask = ~geometry_mask([field_geom_proj], out_shape=field_data.shape, transform=out_transform, invert=False)
                lease_mask = ~geometry_mask([lease_geom_proj], out_shape=field_data.shape, transform=out_transform, invert=False)
                lease_mask = lease_mask & field_mask
                background_mask = field_mask & ~lease_mask
                
                # Extract background pixels
                background_pixels = np.full_like(field_data, np.nan)
                background_pixels[background_mask] = field_data[background_mask]
                
                # Calculate statistics
                stats = calculate_all_stats(background_pixels)
                test_result = test_distribution_normality(stats)
                
                # Store analysis
                analysis = {
                    'filename': raster_file,
                    'stats': stats,
                    'test': test_result,
                    'field_data': field_data,
                    'field_mask': field_mask,
                    'lease_mask': lease_mask,
                    'background_mask': background_mask,
                    'transform': out_transform,
                    'crs': raster_crs
                }
                distribution_analysis.append(analysis)
                
                # Display results
                print(f"📊 Background Statistics:")
                print(f"   Mean: {stats['mean']:.4f}   |  Median: {stats['median']:.4f}")
                print(f"   Std:  {stats['std']:.4f}   |  MAD:    {stats['mad']:.4f}")
                print(f"   Robust Std: {stats['robust_std']:.4f}")
                print(f"\n🎯 Recommendation: {test_result['recommendation'].upper()} Z-SCORE")
                print(f"   Reason: {test_result['reason']}")
                
                if 'warning' in test_result:
                    print(f"   ⚠️ {test_result['warning']}")
                
        except Exception as e:
            print(f"   ❌ Error: {e}")
    
    # Summarize recommendations
    recommendations = [a['test']['recommendation'] for a in distribution_analysis]
    robust_count = recommendations.count('robust')
    normal_count = recommendations.count('normal')
    
    print("\n" + "="*60)
    print("📊 DISTRIBUTION ANALYSIS SUMMARY")
    print("="*60)
    print(f"Files recommending ROBUST z-score: {robust_count}")
    print(f"Files recommending NORMAL z-score: {normal_count}")
    
    if robust_count > normal_count:
        overall_recommendation = 'robust'
        print("\n✅ Overall Recommendation: Use ROBUST Z-SCORE")
        print("   (Better for skewed data and outliers)")
    elif normal_count > robust_count:
        overall_recommendation = 'normal'
        print("\n✅ Overall Recommendation: Use NORMAL Z-SCORE")
        print("   (Better for uniform, normally-distributed data)")
    else:
        overall_recommendation = 'robust'
        print("\n✅ Overall Recommendation: Use ROBUST Z-SCORE")
        print("   (Safer default when recommendations are mixed)")
    
    print("\n💡 Important: Apply the same method to all years for consistency")
else:
    print("❌ Cannot analyze: Missing required files")
    distribution_analysis = []
    overall_recommendation = 'robust'

## 5. Choose Z-Score Method

In [ ]:
# Create interactive selection widget
print("🎯 SELECT Z-SCORE TRANSFORMATION METHOD")
print("="*50)

# Create radio buttons for selection
method_selector = widgets.RadioButtons(
    options=[
        ('Robust Z-Score (Median & MAD) - Resistant to outliers', 'robust'),
        ('Normal Z-Score (Mean & Std Dev) - Better for uniform data', 'normal'),
        (f'Follow Recommendation ({overall_recommendation.upper()})', overall_recommendation)
    ],
    value=overall_recommendation,
    description='Method:',
    disabled=False,
    style={'description_width': 'initial'}
)

# Display comparison table
comparison_html = """
<table style="width:100%; border-collapse: collapse; margin: 10px 0;">
<tr style="background-color: #f0f0f0;">
    <th style="padding: 8px; border: 1px solid #ddd;">Aspect</th>
    <th style="padding: 8px; border: 1px solid #ddd;">Robust Z-Score</th>
    <th style="padding: 8px; border: 1px solid #ddd;">Normal Z-Score</th>
</tr>
<tr>
    <td style="padding: 8px; border: 1px solid #ddd;"><b>Formula</b></td>
    <td style="padding: 8px; border: 1px solid #ddd;">Z = (x - Median) / (1.4826 × MAD)</td>
    <td style="padding: 8px; border: 1px solid #ddd;">Z = (x - Mean) / Std Dev</td>
</tr>
<tr>
    <td style="padding: 8px; border: 1px solid #ddd;"><b>Best For</b></td>
    <td style="padding: 8px; border: 1px solid #ddd;">Skewed data, outliers present</td>
    <td style="padding: 8px; border: 1px solid #ddd;">Normal distribution, uniform data</td>
</tr>
<tr>
    <td style="padding: 8px; border: 1px solid #ddd;"><b>Sensitivity</b></td>
    <td style="padding: 8px; border: 1px solid #ddd;">Less sensitive to extreme values</td>
    <td style="padding: 8px; border: 1px solid #ddd;">More sensitive to all values</td>
</tr>
</table>
"""

display(HTML(comparison_html))
display(method_selector)

# Store the selected method
selected_method = method_selector.value

print("\n📌 Note: The same method will be applied to all rasters for consistency")

## 6. Process NDVI Rasters with Selected Method

In [ ]:
def process_with_selected_method(analysis: Dict, method: str) -> Dict[str, Any]:
    """Process a raster with the selected z-score method"""
    
    field_data = analysis['field_data']
    field_mask = analysis['field_mask']
    lease_mask = analysis['lease_mask']
    stats = analysis['stats']
    
    # Initialize z-score raster with NaN
    z_score_raster = np.full(field_data.shape, np.nan, dtype=np.float32)
    
    # Find valid field pixels
    valid_field_pixels = field_mask & ~np.isnan(field_data)
    nodata_values = [-9999, -10000, -3.4028235e+38]
    for ndv in nodata_values:
        valid_field_pixels = valid_field_pixels & (np.abs(field_data - ndv) > 1e-6)
    
    # Calculate z-scores based on selected method
    if method == 'robust':
        if stats['robust_std'] > 0:
            z_score_raster[valid_field_pixels] = (
                field_data[valid_field_pixels] - stats['median']
            ) / stats['robust_std']
        center_value = stats['median']
        spread_value = stats['robust_std']
        method_label = 'Robust Z-Score (Median & MAD)'
    else:  # normal
        if stats['std'] > 0:
            z_score_raster[valid_field_pixels] = (
                field_data[valid_field_pixels] - stats['mean']
            ) / stats['std']
        center_value = stats['mean']
        spread_value = stats['std']
        method_label = 'Normal Z-Score (Mean & Std Dev)'
    
    # Ensure everything outside field boundary is NaN
    z_score_raster[~field_mask] = np.nan
    
    # Calculate lease statistics
    lease_pixels = z_score_raster[lease_mask & ~np.isnan(z_score_raster)]
    
    if len(lease_pixels) > 0:
        lease_stats = {
            'mean_z': np.mean(lease_pixels),
            'median_z': np.median(lease_pixels),
            'std_z': np.std(lease_pixels),
            'min_z': np.min(lease_pixels),
            'max_z': np.max(lease_pixels),
            'n_pixels': len(lease_pixels),
            'pct_above_zero': np.sum(lease_pixels > 0) / len(lease_pixels) * 100,
            'pct_below_minus2': np.sum(lease_pixels < -2) / len(lease_pixels) * 100,
            'pct_above_2': np.sum(lease_pixels > 2) / len(lease_pixels) * 100
        }
    else:
        lease_stats = None
    
    return {
        'filename': analysis['filename'],
        'z_score_raster': z_score_raster,
        'transform': analysis['transform'],
        'crs': analysis['crs'],
        'stats': stats,
        'lease_stats': lease_stats,
        'method': method,
        'method_label': method_label,
        'center_value': center_value,
        'spread_value': spread_value,
        'lease_mask': lease_mask,
        'field_mask': field_mask,
        'background_mask': analysis['background_mask'],
        'shape': z_score_raster.shape,
        'success': True
    }

# Process all rasters with selected method
processed_rasters = []
selected_method = method_selector.value

if distribution_analysis:
    print(f"\n🔄 Processing with {selected_method.upper()} Z-SCORE method...")
    print("="*60)
    
    for i, analysis in enumerate(distribution_analysis):
        print(f"\n[{i+1}/{len(distribution_analysis)}] {analysis['filename']}")
        
        result = process_with_selected_method(analysis, selected_method)
        processed_rasters.append(result)
        
        print(f"✅ Processed using {result['method_label']}")
        print(f"   Center: {result['center_value']:.4f}")
        print(f"   Spread: {result['spread_value']:.4f}")
        
        if result['lease_stats']:
            lease_stats = result['lease_stats']
            print(f"\n   Lease Area Z-Scores:")
            print(f"   • Mean: {lease_stats['mean_z']:.3f}")
            print(f"   • Above background (Z>0): {lease_stats['pct_above_zero']:.1f}%")
            
            # Interpretation
            if lease_stats['mean_z'] > 0.5:
                print("   ✅ Lease performing ABOVE background")
            elif lease_stats['mean_z'] > -0.5:
                print("   ⚠️ Lease performing SIMILAR to background")
            else:
                print("   ❌ Lease performing BELOW background")
    
    print(f"\n✅ Processed {len(processed_rasters)} rasters with {selected_method.upper()} method")
else:
    print("❌ No data to process")

## 7. Visualize Results

In [ ]:
def plot_z_score_results(result: Dict, figsize=(15, 8)):
    """Create comprehensive visualization of z-score results"""
    
    fig, axes = plt.subplots(1, 3, figsize=figsize)
    
    z_raster = result['z_score_raster']
    lease_mask = result['lease_mask']
    field_mask = result['field_mask']
    background_mask = result['background_mask']
    
    # Create custom colormap
    colors = ['darkred', 'red', 'white', 'lightgreen', 'darkgreen']
    cmap = mcolors.LinearSegmentedColormap.from_list('z_score', colors, N=100)
    vmin, vmax = -3, 3
    
    # Plot 1: Z-score map
    im1 = axes[0].imshow(z_raster, cmap=cmap, vmin=vmin, vmax=vmax)
    axes[0].set_title(f'Z-Score Map\n({result["method_label"]})', fontweight='bold')
    axes[0].axis('off')
    
    # Add lease boundary overlay
    lease_overlay = np.ma.masked_where(~lease_mask, np.ones_like(z_raster))
    axes[0].imshow(lease_overlay, alpha=0.2, cmap='Blues')
    
    # Plot 2: Area masks
    mask_display = np.zeros_like(z_raster)
    mask_display[background_mask] = 1
    mask_display[lease_mask] = 2
    mask_display[~field_mask] = np.nan
    
    cmap_masks = mcolors.ListedColormap(['lightgray', 'lightblue'])
    axes[1].imshow(mask_display, cmap=cmap_masks, alpha=0.8)
    axes[1].set_title('Area Definitions', fontweight='bold')
    axes[1].axis('off')
    
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='lightgray', label='Background'),
        Patch(facecolor='lightblue', label='Lease Area')
    ]
    axes[1].legend(handles=legend_elements, loc='upper right')
    
    # Plot 3: Distribution histogram
    lease_z = z_raster[lease_mask & ~np.isnan(z_raster)]
    background_z = z_raster[background_mask & ~np.isnan(z_raster)]
    
    axes[2].hist(background_z, bins=50, alpha=0.5, label='Background', color='gray', density=True)
    axes[2].hist(lease_z, bins=50, alpha=0.7, label='Lease Area', color='blue', density=True)
    axes[2].axvline(0, color='black', linestyle='--', linewidth=2, label='Background Center')
    axes[2].axvline(-2, color='red', linestyle=':', alpha=0.5)
    axes[2].axvline(2, color='red', linestyle=':', alpha=0.5, label='±2 Std')
    
    axes[2].set_xlabel('Z-Score', fontsize=10)
    axes[2].set_ylabel('Density', fontsize=10)
    axes[2].set_title('Z-Score Distributions', fontweight='bold')
    axes[2].legend(loc='upper right')
    axes[2].grid(True, alpha=0.3)
    axes[2].set_xlim(-4, 4)
    
    # Add colorbar
    cbar = plt.colorbar(im1, ax=axes, orientation='horizontal', pad=0.1, aspect=40)
    cbar.set_label(f'Z-Score ({result["method"].capitalize()} Method)', fontsize=10)
    
    # Add title
    fig.suptitle(f"File: {result['filename']}", fontsize=13, fontweight='bold', y=1.02)
    
    plt.tight_layout()
    return fig

# Visualize results
if processed_rasters:
    print("\n📊 Generating visualizations...")
    
    # Show first few rasters
    max_plots = min(3, len(processed_rasters))
    
    for i in range(max_plots):
        print(f"\nVisualization {i+1}/{max_plots}")
        fig = plot_z_score_results(processed_rasters[i])
        plt.show()
    
    if len(processed_rasters) > max_plots:
        print(f"\n📌 Showing first {max_plots} of {len(processed_rasters)} visualizations")

## 8. Export Results & Download

In [ ]:
# Save processed rasters and create download package
if processed_rasters:
    output_dir = f'zscore_{selected_method}_outputs'
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"💾 Saving {selected_method.upper()} z-score transformed rasters...")
    print("="*50)
    
    saved_files = []
    summary_data = []
    
    for i, result in enumerate(processed_rasters):
        # Save GeoTIFF
        base_name = os.path.splitext(result['filename'])[0]
        output_file = os.path.join(output_dir, f"{base_name}_zscore_{selected_method}.tif")
        
        try:
            z_data = result['z_score_raster'].copy()
            
            with rasterio.open(
                output_file,
                'w',
                driver='GTiff',
                height=result['shape'][0],
                width=result['shape'][1],
                count=1,
                dtype='float32',
                crs=result['crs'],
                transform=result['transform'],
                compress='lzw',
                nodata=-9999
            ) as dst:
                z_data[np.isnan(z_data)] = -9999
                dst.write(z_data.astype(np.float32), 1)
                
                # Add metadata
                dst.update_tags(
                    method=result['method'],
                    center_value=str(result['center_value']),
                    spread_value=str(result['spread_value']),
                    processing_date=datetime.now().isoformat()
                )
            
            saved_files.append(output_file)
            print(f"   ✅ Saved: {os.path.basename(output_file)}")
            
            # Collect summary data
            row = {
                'Filename': result['filename'],
                'Method': result['method'],
                'Center': result['center_value'],
                'Spread': result['spread_value']
            }
            
            if result['lease_stats']:
                row.update({
                    'Lease_Mean_Z': result['lease_stats']['mean_z'],
                    'Lease_Pct_Above_Background': result['lease_stats']['pct_above_zero']
                })
            
            summary_data.append(row)
            
        except Exception as e:
            print(f"   ❌ Error saving: {e}")
    
    # Save summary CSV
    df_summary = pd.DataFrame(summary_data)
    summary_file = os.path.join(output_dir, f'zscore_{selected_method}_summary.csv')
    df_summary.to_csv(summary_file, index=False)
    
    # Create ZIP archive
    zip_filename = f'zscore_{selected_method}_results.zip'
    
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in saved_files:
            if os.path.exists(file):
                zipf.write(file, os.path.basename(file))
        
        if os.path.exists(summary_file):
            zipf.write(summary_file, os.path.basename(summary_file))
        
        # Add README with styling guide
        readme_content = f"""Z-Score Transformed NDVI - {selected_method.upper()} Method
="*60

Processing Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}
Method Used: {selected_method.upper()} Z-SCORE

QGIS STYLING RECOMMENDATIONS
-----------------------------

For Individual Year Z-Scores (5 classes):
1. Class 1: < -2 (Dark Red) - Significantly below background
2. Class 2: -2 to -1 (Light Red) - Moderately below background
3. Class 3: -1 to 0 (Yellow/White) - Slightly below background
4. Class 4: 0 to 1 (Light Green) - Slightly above background
5. Class 5: > 1 (Dark Green) - Above background

For Multi-Year Composite Analysis:
- If seeing saturation on negatives, try:
  • Adjust min value to -5 or -10 instead of default -3
  • Use percentile-based classification (2%, 25%, 50%, 75%, 98%)
  • Apply histogram equalization for better contrast

NoData Value: -9999

Files included:
- *_zscore_{selected_method}.tif: Transformed rasters
- zscore_{selected_method}_summary.csv: Statistics summary
"""
        
        zipf.writestr('README.txt', readme_content)
    
    print(f"\n✅ Archive created: {zip_filename}")
    print("\n⬇️ Starting download...")
    
    # Trigger download
    files.download(zip_filename)
    
    print("\n🎉 Processing complete!")
    print(f"   Method used: {selected_method.upper()} Z-SCORE")
    print(f"   Files processed: {len(processed_rasters)}")
else:
    print("⚠️ No files to download")